In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("Ayurveda.pdf")
docs=loader.load()

In [12]:
docs

[Document(page_content='', metadata={'source': 'Ayurveda.pdf', 'page': 0}),
 Document(page_content='ALSO BY THE AUTHOR\nAyurveda Cooking for Self-Healing\n (with Usha Lad)\nThe Yoga of Herbs\n (with David Frawley)\nAyurveda: The Science of Self-Healing\nSecrets of the Pulse', metadata={'source': 'Ayurveda.pdf', 'page': 1}),
 Document(page_content='', metadata={'source': 'Ayurveda.pdf', 'page': 2}),
 Document(page_content='Although the information contained in this book is based on Ayurvedic principles practiced for thousands of years, it\nshould not be taken or construed as standard medical diagnosis or treatment. For any medical condition, always\nconsult with a qualified physician.\nCopyright © 1998 by Vasant Lad, M.A.Sc.\nAll rights reserved. No part of this book may be reproduced or transmitted in any form or by any means, electronic or\nmechanical, including photocopying, recording, or by any information storage and retrieval system, without permission in\nwriting from the publish

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_API_KEY"]=str(os.getenv("LANGCHAIN_API_KEY"))

In [76]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents[0:5]


[Document(page_content='ALSO BY THE AUTHOR\nAyurveda Cooking for Self-Healing\n (with Usha Lad)\nThe Yoga of Herbs\n (with David Frawley)\nAyurveda: The Science of Self-Healing\nSecrets of the Pulse', metadata={'source': 'Ayurveda.pdf', 'page': 1}),
 Document(page_content='Although the information contained in this book is based on Ayurvedic principles practiced for thousands of years, it\nshould not be taken or construed as standard medical diagnosis or treatment. For any medical condition, always\nconsult with a qualified physician.\nCopyright © 1998 by Vasant Lad, M.A.Sc.\nAll rights reserved. No part of this book may be reproduced or transmitted in any form or by any means, electronic or\nmechanical, including photocopying, recording, or by any information storage and retrieval system, without permission in\nwriting from the publisher.\nPublished by Three Rivers Press, New York, New York.\nMember of the Crown Publishing Group.\nRandom House, Inc. New York, Toronto, London, Sydney, 

In [32]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db1=Chroma.from_documents(documents,OllamaEmbeddings(model='llama3'))

In [105]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
DB=Chroma.from_documents(documents[0:20],OllamaEmbeddings(model='llama3'),persist_directory='DB')

In [39]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db2=FAISS.from_documents(documents[0:20],OllamaEmbeddings(model='llama3'))

In [104]:
db2.save_local("FDB")

In [75]:
query="“the science of life and longevity"
result=db1.similarity_search(query)
result

[Document(page_content='Part I\n \n \nThe Science of Life', metadata={'page': 13, 'source': 'Ayurveda.pdf'}),
 Document(page_content='and swelling of the extremities; proteinurea, convulsions, and coma can develop.\nThis dangerous condition is called eclampsia or pregnancy toxemia. It cannot be\ntreated by home remedies; it requires prompt, expert medical supervision and\ncare.\nHowever, the following herbal formula will help, as an adjunct to treatment,\nto relieve the convulsions:\nbrahmi\njatamamsi\nshanka pushpi\nMix these herbs in equal amounts. Use ½ teaspoon of the mixture to make a\ntea, and drink it twice a day.\nNOTE\n: If there is ever a generalized swelling of the entire body (including, in men, a\nswelling of the penis and scrotum), this is a very serious, life-threatening condition that\nrequires immediate treatment.\nFOR HIGH BLOOD PRESSURE\n. Use these herbs:\nhawthorn berry\npassion flower\narjuna\nMix in equal proportion, and use 1 teaspoon to make a tea. Take twice a

In [61]:
result[3].page_content

'create, maintain, or restore health and balance.\nAyurveda\n is a Sanskrit word that means “the science of life and longevity.” According\nto this science, every individual is both a creation of cosmic energies and a unique\nphenomenon, a unique personality. Ayurveda teaches that we all have a constitution,\nwhich is our individual psychobiological makeup. From the moment of conception, this\nindividual constitution is created by the universal energies of Space, Air, Fire, Water,\nand Earth.\nThese ɹve elements combine into the three fundamental energies, or \ndoshas\n. Ether\nand air constitute vata, which is the energy of movement; ɹre and water constitute\npitta, the principle of digestion or metabolism, the transformation of matter into energy;\nand water and earth make up kapha, the energy of structure and lubrication. When the\nmale sperm and the female egg join at the time of fertilization, the vata–pitta–kapha'

In [44]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [86]:
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Rephrase the text found from the context into a good paragraph
<context>
{context}
<content>
Question:{input}                                     
 """)

In [87]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [88]:
retriver=db1.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000016760DBC350>)

In [112]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser(output_keys=["answer"])

In [115]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriver,document_chain)

In [116]:
response=retrieval_chain.invoke({"input":"“What precautions need to be taken during pregnancy?"})

In [95]:
response['answer']

"I'd be happy to help!\n\nBased on the provided context, here's an answer to your question:\n\nDuring pregnancy, it is essential to maintain good digestion and overall health. Some key precautions include avoiding emotional eating, as this can affect digestion negatively. Eat only two or three meals a day, with small sips of warm water during meals, rather than snacking in between. Make sure to chew food well, as saliva plays a crucial role in digestion.\n\nAdditionally, consider drinking a cup of lassi after meals, made by blending yogurt, ginger, and cumin powder with water. This can help regulate blood pressure. Also, practice yoga postures that aid digestion, such as Leg Lifts and the Peacock pose.\n\nFurthermore, take care of your teeth and gums by maintaining good oral hygiene practices, and consider applying tea tree oil or clove oil to relieve toothaches. It's crucial to get any cavities treated and follow Ayurvedic recommendations for healthy teeth and gums.\n\nLastly, if you 

In [117]:
parsed_response = output_parser.parse(response)

# Now, the response will be in the format defined by the StringOutputParser
Answer = parsed_response["answer"]

In [118]:
Answer

'Based on the provided context, I\'ll rephrase the text into a good paragraph:\n\nDuring pregnancy, it\'s essential to maintain good digestion and oral health. Avoid emotional eating, as it can negatively impact digestion. Instead, stick to three meals a day - breakfast, lunch, and dinner - and avoid snacking in between. When drinking, opt for warm water instead of cold or iced drinks, which can cool the digestive fire. Make sure to chew your food well and leave some space in your stomach for proper digestion. You can also consider finishing your meal with a cup of lassi, made by blending yogurt, ginger, and cumin powder with water. Additionally, certain yoga postures like Leg Lifts and Peacock pose can be helpful for improving digestion during pregnancy.\n\nNote that I did not answer the question "What precautions need to be taken during pregnancy?" as it was not directly related to the provided context.'

In [122]:
import textwrap


# Set the maximum width of each line to 20 words
wrapper = textwrap.TextWrapper(width=100)

# Wrap the output string to limit the number of words per line
wrapped_precautions = wrapper.fill(Answer)

# Print the wrapped output
print(wrapped_precautions)

Based on the provided context, I'll rephrase the text into a good paragraph:  During pregnancy, it's
essential to maintain good digestion and oral health. Avoid emotional eating, as it can negatively
impact digestion. Instead, stick to three meals a day - breakfast, lunch, and dinner - and avoid
snacking in between. When drinking, opt for warm water instead of cold or iced drinks, which can
cool the digestive fire. Make sure to chew your food well and leave some space in your stomach for
proper digestion. You can also consider finishing your meal with a cup of lassi, made by blending
yogurt, ginger, and cumin powder with water. Additionally, certain yoga postures like Leg Lifts and
Peacock pose can be helpful for improving digestion during pregnancy.  Note that I did not answer
the question "What precautions need to be taken during pregnancy?" as it was not directly related to
the provided context.
